In [1]:
# UK Energy Grid Analysis
# Exploring electricity generation patterns, renewable energy trends, and grid dynamics

import sys
sys.path.insert(0, '../')

# Import our production code
from src.data.fetch_data import ElexonDataFetcher

# Data libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Settings
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

Libraries loaded
Analysis date: 2026-01-19


In [15]:
# Fetch 30 days of historical data for comprehensive analysis
print("Fetching 30 days of UK electricity generation data...")

fetcher = ElexonDataFetcher()
df = fetcher.fetch_historical_data(days=30)

print(f"\n📊 Dataset Overview:")
print(f"  Records: {len(df):,}")
print(f"  Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"  Fuel types: {df['fuel_type'].nunique()}")
print(f"  Time periods: {df['timestamp'].nunique()}")

# Quick peek at the data
print(f"\n🔍 Sample data:")
display(df.head())

# Save for later use
df.to_csv('../data/processed/uk_generation_30days.csv', index=False)
print("\nData fetched and saved")

INFO:src.data.fetch_data:Fetching chunk: 2025-12-20 to 2025-12-27
INFO:src.data.fetch_data:Fetching generation data from 2025-12-20 15:57:22.285291 to 2025-12-27 15:57:22.285291
INFO:src.data.fetch_data:API request completed in 125.74ms - Status: 200
INFO:src.data.fetch_data:Successfully fetched 3696 records


Fetching 30 days of UK electricity generation data...


INFO:src.data.fetch_data:Fetching chunk: 2025-12-27 to 2026-01-03
INFO:src.data.fetch_data:Fetching generation data from 2025-12-27 15:57:22.285291 to 2026-01-03 15:57:22.285291
INFO:src.data.fetch_data:API request completed in 77.36ms - Status: 200
INFO:src.data.fetch_data:Successfully fetched 3696 records
INFO:src.data.fetch_data:Fetching chunk: 2026-01-03 to 2026-01-10
INFO:src.data.fetch_data:Fetching generation data from 2026-01-03 15:57:22.285291 to 2026-01-10 15:57:22.285291
INFO:src.data.fetch_data:API request completed in 65.41ms - Status: 200
INFO:src.data.fetch_data:Successfully fetched 3696 records
INFO:src.data.fetch_data:Fetching chunk: 2026-01-10 to 2026-01-17
INFO:src.data.fetch_data:Fetching generation data from 2026-01-10 15:57:22.285291 to 2026-01-17 15:57:22.285291
INFO:src.data.fetch_data:API request completed in 67.06ms - Status: 200
INFO:src.data.fetch_data:Successfully fetched 3201 records
INFO:src.data.fetch_data:Fetching chunk: 2026-01-17 to 2026-01-19
INFO:sr


📊 Dataset Overview:
  Records: 11,704
  Date range: 2025-12-20 16:00:00+00:00 to 2026-01-19 14:00:00+00:00
  Fuel types: 11
  Time periods: 1064

🔍 Sample data:


,timestamp,fuel_type,generation_mw,total_generation
0,2025-12-20 16:00:00+00:00,Biomass,2403.0,33906.012
1,2025-12-20 16:00:00+00:00,Fossil Gas,13553.0,33906.012
2,2025-12-20 16:00:00+00:00,Fossil Hard coal,0.0,33906.012
3,2025-12-20 16:00:00+00:00,Fossil Oil,0.0,33906.012
4,2025-12-20 16:00:00+00:00,Hydro Pumped Storage,910.0,33906.012



Data fetched and saved


In [16]:
# Prepare data for visualization
df_pivot = df.pivot_table(
    index='timestamp',
    columns='fuel_type', 
    values='generation_mw',
    aggfunc='sum'
).fillna(0)

# Calculate totals and renewable percentage
renewables = ['Solar', 'Wind Offshore', 'Wind Onshore', 
              'Hydro Run-of-river and poundage', 'Hydro Pumped Storage', 'Biomass']

df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot['Renewables'] = df_pivot[renewables].sum(axis=1)
df_pivot['Renewable_Pct'] = (df_pivot['Renewables'] / df_pivot['Total']) * 100

# Add time features for analysis
df_pivot['Date'] = df_pivot.index.date
df_pivot['Hour'] = df_pivot.index.hour
df_pivot['Day_of_Week'] = df_pivot.index.day_name()

print(f"📈 Summary Statistics (30 days):")
print(f"  Average total generation: {df_pivot['Total'].mean():,.0f} MW")
print(f"  Average renewable %: {df_pivot['Renewable_Pct'].mean():.1f}%")
print(f"  Peak renewable %: {df_pivot['Renewable_Pct'].max():.1f}%")
print(f"  Lowest renewable %: {df_pivot['Renewable_Pct'].min():.1f}%")

print(f"\n⚡ Generation by source (average MW):")
for fuel in df_pivot.columns[:11]:  # First 11 are fuel types
    print(f"  {fuel:30s}: {df_pivot[fuel].mean():8,.0f} MW")

📈 Summary Statistics (30 days):
  Average total generation: 34,344 MW
  Average renewable %: 49.6%
  Peak renewable %: 77.1%
  Lowest renewable %: 18.4%

⚡ Generation by source (average MW):
  Biomass                       :    2,557 MW
  Fossil Gas                    :   13,085 MW
  Fossil Hard coal              :        0 MW
  Fossil Oil                    :        0 MW
  Hydro Pumped Storage          :      231 MW
  Hydro Run-of-river and poundage:      550 MW
  Nuclear                       :    3,905 MW
  Other                         :      669 MW
  Solar                         :      741 MW
  Wind Offshore                 :    8,279 MW
  Wind Onshore                  :    4,327 MW


In [17]:
# Visualization 1: 30-Day Generation Mix (Stacked Area Chart)

# Define professional color scheme
colors = {
    'Nuclear': '#E74C3C',           # Red
    'Fossil Gas': '#95A5A6',        # Gray
    'Fossil Hard coal': '#34495E',  # Dark gray
    'Wind Offshore': '#3498DB',     # Blue
    'Wind Onshore': '#5DADE2',      # Light blue
    'Solar': '#F39C12',             # Orange
    'Biomass': '#27AE60',           # Green
    'Hydro Run-of-river and poundage': '#16A085', # Teal
    'Hydro Pumped Storage': '#1ABC9C', # Light teal
    'Other': '#7F8C8D',             # Light gray
    'Fossil Oil': '#2C3E50'         # Very dark gray
}

# Order for stacking (fossil at bottom, renewables on top)
fuel_order = [
    'Fossil Hard coal', 'Fossil Oil', 'Fossil Gas', 'Nuclear', 'Other',
    'Biomass', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage',
    'Solar', 'Wind Onshore', 'Wind Offshore'
]

fig = go.Figure()

# Add traces in order
for fuel_type in fuel_order:
    if fuel_type in df_pivot.columns:
        fig.add_trace(go.Scatter(
            x=df_pivot.index,
            y=df_pivot[fuel_type],
            name=fuel_type,
            mode='lines',
            stackgroup='one',
            fillcolor=colors.get(fuel_type, '#95A5A6'),
            line=dict(width=0.5, color=colors.get(fuel_type, '#95A5A6')),
            hovertemplate='<b>%{fullData.name}</b><br>%{y:,.0f} MW<extra></extra>'
        ))

fig.update_layout(
    title={
        'text': 'UK Electricity Generation Mix - Last 30 Days',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'family': 'Arial, sans-serif'}
    },
    xaxis_title='Date',
    yaxis_title='Generation (MW)',
    hovermode='x unified',
    height=600,
    width=1400,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.02,
        font=dict(size=11)
    ),
    margin=dict(r=150)
)

fig.show()

In [18]:
# Visualization 2: Renewable Energy Percentage Trend

fig = go.Figure()

# Add renewable percentage line
fig.add_trace(go.Scatter(
    x=df_pivot.index,
    y=df_pivot['Renewable_Pct'],
    mode='lines',
    name='Renewable %',
    line=dict(color='#27AE60', width=2.5),
    fill='tozeroy',
    fillcolor='rgba(39, 174, 96, 0.2)',
    hovertemplate='<b>Renewable Energy</b><br>%{y:.1f}%<extra></extra>'
))

# Add mean line
mean_renewable = df_pivot['Renewable_Pct'].mean()
fig.add_hline(
    y=mean_renewable,
    line_dash="dash",
    line_color="red",
    annotation_text=f"30-day average: {mean_renewable:.1f}%",
    annotation_position="top right"
)

# Add 50% target line
fig.add_hline(
    y=50,
    line_dash="dot",
    line_color="gray",
    annotation_text="50% threshold",
    annotation_position="bottom right"
)

fig.update_layout(
    title={
        'text': 'UK Renewable Energy Percentage - 30 Day Trend',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'family': 'Arial, sans-serif'}
    },
    xaxis_title='Date',
    yaxis_title='Renewable Energy (%)',
    height=600,
    width=1400,
    template='plotly_white',
    hovermode='x unified',
    yaxis=dict(range=[0, 100]),
    margin=dict(l=80, r=80, t=100, b=80)
)

fig.show()

print(f"\n📊 Renewable Energy Insights:")
print(f"  • UK exceeded 50% renewable for {(df_pivot['Renewable_Pct'] > 50).sum()} periods ({(df_pivot['Renewable_Pct'] > 50).sum() / len(df_pivot) * 100:.1f}% of the time)")
print(f"  • Highest renewable day: {df_pivot.groupby('Date')['Renewable_Pct'].mean().idxmax()} ({df_pivot.groupby('Date')['Renewable_Pct'].mean().max():.1f}%)")
print(f"  • Volatility (std dev): {df_pivot['Renewable_Pct'].std():.1f}%")


📊 Renewable Energy Insights:
  • UK exceeded 50% renewable for 524 periods (49.2% of the time)
  • Highest renewable day: 2026-01-01 (69.1%)
  • Volatility (std dev): 14.2%


In [19]:
# Visualization 3: Average Generation by Fuel Type (30 days)

# Calculate average generation for each fuel type
fuel_averages = df_pivot[fuel_order].mean().sort_values(ascending=False)

# Group into categories
categories = {
    'Wind (Offshore + Onshore)': fuel_averages[['Wind Offshore', 'Wind Onshore']].sum(),
    'Gas': fuel_averages['Fossil Gas'],
    'Nuclear': fuel_averages['Nuclear'],
    'Biomass': fuel_averages['Biomass'],
    'Solar': fuel_averages['Solar'],
    'Hydro (All)': fuel_averages[['Hydro Pumped Storage', 'Hydro Run-of-river and poundage']].sum(),
    'Other': fuel_averages['Other'],
    'Coal': fuel_averages['Fossil Hard coal'],
    'Oil': fuel_averages['Fossil Oil']
}

# Remove zeros
categories = {k: v for k, v in categories.items() if v > 0}

# Create donut chart
fig = go.Figure(data=[go.Pie(
    labels=list(categories.keys()),
    values=list(categories.values()),
    hole=0.4,
    marker=dict(
        colors=['#3498DB', '#95A5A6', '#E74C3C', '#27AE60', 
                '#F39C12', '#16A085', '#7F8C8D'],
        line=dict(color='white', width=2)
    ),
    textinfo='label+percent',
    textfont=dict(size=14),
    hovertemplate='<b>%{label}</b><br>%{value:,.0f} MW<br>%{percent}<extra></extra>'
)])

fig.update_layout(
    title={
        'text': 'UK Average Generation Mix - Last 30 Days',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'family': 'Arial, sans-serif'}
    },
    height=700,
    width=1000,
    template='plotly_white',
    annotations=[dict(
        text=f'{df_pivot["Total"].mean():,.0f}<br>MW<br>Average',
        x=0.5, y=0.5,
        font_size=20,
        showarrow=False
    )],
    margin=dict(l=80, r=80, t=100, b=80)
)

fig.show()

print(f"\n⚡ Generation Breakdown:")
for fuel, mw in sorted(categories.items(), key=lambda x: x[1], reverse=True):
    pct = (mw / df_pivot['Total'].mean()) * 100
    print(f"  {fuel:30s}: {mw:8,.0f} MW ({pct:5.1f}%)")


⚡ Generation Breakdown:
  Gas                           :   13,085 MW ( 38.1%)
  Wind (Offshore + Onshore)     :   12,606 MW ( 36.7%)
  Nuclear                       :    3,905 MW ( 11.4%)
  Biomass                       :    2,557 MW (  7.4%)
  Hydro (All)                   :      781 MW (  2.3%)
  Solar                         :      741 MW (  2.2%)
  Other                         :      669 MW (  1.9%)


In [20]:
# Visualization 4: Renewable Percentage by Hour and Day

# Create pivot for heatmap
heatmap_data = df_pivot.pivot_table(
    index='Hour',
    columns='Day_of_Week',
    values='Renewable_Pct',
    aggfunc='mean'
)

# Reorder days of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data[[day for day in day_order if day in heatmap_data.columns]]

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlGn',  # Red (low) to Green (high)
    text=heatmap_data.values.round(1),
    texttemplate='%{text}%',
    textfont={"size": 10},
    colorbar=dict(title="Renewable %"),
    hovertemplate='<b>%{x}</b><br>Hour: %{y}:00<br>Renewable: %{z:.1f}%<extra></extra>'
))

fig.update_layout(
    title={
        'text': 'Renewable Energy by Time of Day and Day of Week',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'family': 'Arial, sans-serif'}
    },
    xaxis_title='Day of Week',
    yaxis_title='Hour of Day',
    height=700,
    width=1200,
    template='plotly_white',
    yaxis=dict(autorange='reversed'),
    margin=dict(l=80, r=80, t=100, b=80)
)

fig.show()

print(f"\n🕐 Time Pattern Insights:")
print(f"  • Peak renewable hour: {df_pivot.groupby('Hour')['Renewable_Pct'].mean().idxmax()}:00 ({df_pivot.groupby('Hour')['Renewable_Pct'].mean().max():.1f}%)")
print(f"  • Lowest renewable hour: {df_pivot.groupby('Hour')['Renewable_Pct'].mean().idxmin()}:00 ({df_pivot.groupby('Hour')['Renewable_Pct'].mean().min():.1f}%)")
print(f"  • Most renewable day: {df_pivot.groupby('Day_of_Week')['Renewable_Pct'].mean().idxmax()} ({df_pivot.groupby('Day_of_Week')['Renewable_Pct'].mean().max():.1f}%)")




🕐 Time Pattern Insights:
  • Peak renewable hour: 2:00 (57.3%)
  • Lowest renewable hour: 16:00 (45.0%)
  • Most renewable day: Saturday (55.7%)


In [21]:
# Create comprehensive dashboard with all visualizations

from plotly.subplots import make_subplots

# Create subplot layout
fig = make_subplots(
    rows=3, cols=2,
    row_heights=[0.35, 0.35, 0.3],
    column_widths=[0.7, 0.3],
    specs=[
        [{"type": "scatter", "colspan": 2}, None],  # Row 1: Full width stacked area
        [{"type": "scatter"}, {"type": "pie"}],      # Row 2: Line chart + Donut
        [{"type": "heatmap", "colspan": 2}, None]    # Row 3: Full width heatmap
    ],
    subplot_titles=(
        'Generation Mix Over Time (30 Days)',
        '',
        'Renewable Energy Trend',
        'Average Fuel Mix',
        'Renewable % by Hour and Day'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Stacked Area Chart (Row 1)
for fuel_type in fuel_order:
    if fuel_type in df_pivot.columns:
        fig.add_trace(go.Scatter(
            x=df_pivot.index,
            y=df_pivot[fuel_type],
            name=fuel_type,
            mode='lines',
            stackgroup='one',
            fillcolor=colors.get(fuel_type, '#95A5A6'),
            line=dict(width=0.5),
            legendgroup='fuel',
            showlegend=True,
            hovertemplate='%{y:,.0f} MW<extra></extra>'
        ), row=1, col=1)

# 2. Renewable Percentage Line (Row 2, Left)
fig.add_trace(go.Scatter(
    x=df_pivot.index,
    y=df_pivot['Renewable_Pct'],
    mode='lines',
    name='Renewable %',
    line=dict(color='#27AE60', width=2.5),
    fill='tozeroy',
    fillcolor='rgba(39, 174, 96, 0.2)',
    legendgroup='renewable',
    showlegend=False,
    hovertemplate='%{y:.1f}%<extra></extra>'
), row=2, col=1)

# Add mean line to renewable chart
mean_renewable = df_pivot['Renewable_Pct'].mean()
fig.add_hline(
    y=mean_renewable,
    line_dash="dash",
    line_color="red",
    row=2, col=1,
    annotation_text=f"Avg: {mean_renewable:.1f}%",
    annotation_position="top right"
)

# 3. Donut Chart (Row 2, Right)
fig.add_trace(go.Pie(
    labels=list(categories.keys()),
    values=list(categories.values()),
    hole=0.5,
    marker=dict(
        colors=['#3498DB', '#95A5A6', '#E74C3C', '#27AE60', 
                '#F39C12', '#16A085', '#7F8C8D'],
        line=dict(color='white', width=2)
    ),
    textinfo='label+percent',
    textfont=dict(size=11),
    legendgroup='pie',
    showlegend=False,
    hovertemplate='<b>%{label}</b><br>%{value:,.0f} MW<extra></extra>'
), row=2, col=2)

# 4. Heatmap (Row 3)
fig.add_trace(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlGn',
    text=heatmap_data.values.round(1),
    texttemplate='%{text}%',
    textfont={"size": 9},
    colorbar=dict(title="Renewable %", len=0.3, y=0.15),
    legendgroup='heatmap',
    showlegend=False,
    hovertemplate='%{x}<br>Hour: %{y}:00<br>%{z:.1f}%<extra></extra>'
), row=3, col=1)

# Update layout
fig.update_layout(
    title={
        'text': f'UK Energy Grid Dashboard - 30 Day Analysis<br><sub>Data from {df["timestamp"].min().strftime("%Y-%m-%d")} to {df["timestamp"].max().strftime("%Y-%m-%d")}</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 28, 'family': 'Arial, sans-serif'}
    },
    height=1400,
    width=1600,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=1.15,
        font=dict(size=10)
    ),
    margin=dict(l=80, r=250, t=150, b=80)
)

# Update axes labels
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Generation (MW)", row=1, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Renewable %", row=2, col=1, range=[0, 100])

fig.update_xaxes(title_text="Day of Week", row=3, col=1)
fig.update_yaxes(title_text="Hour", row=3, col=1, autorange='reversed')

fig.show()

print("Comprehensive dashboard created!")
print(f"\n📊 Dashboard Summary:")
print(f"  • Total periods analyzed: {len(df_pivot):,}")
print(f"  • Average generation: {df_pivot['Total'].mean():,.0f} MW")
print(f"  • Average renewable: {df_pivot['Renewable_Pct'].mean():.1f}%")
print(f"  • Wind dominates renewables: {(fuel_averages[['Wind Offshore', 'Wind Onshore']].sum() / df_pivot['Renewables'].mean() * 100):.1f}%")

Comprehensive dashboard created!

📊 Dashboard Summary:
  • Total periods analyzed: 1,064
  • Average generation: 34,344 MW
  • Average renewable: 49.6%
  • Wind dominates renewables: 75.6%


### **Key Findings: UK Energy Grid Analysis (30 Days)**

#### **🌍 The Big Picture**
- **Average renewable generation: 49.6%** - UK is approaching a renewable-majority grid
- **Peak renewable achieved: 77.1%** on windy days
- **Wind is now competitive with gas**: Wind (36.7%) vs Gas (38.1%)
- **Coal completely phased out**: 0 MW generated over the entire 30-day period

#### **⚡ Generation Mix**
| Source | Average MW | % of Total |
|--------|------------|------------|
| Gas | 13,085 | 38.1% |
| **Wind (Combined)** | **12,606** | **36.7%** |
| Nuclear | 3,905 | 11.4% |
| Biomass | 2,557 | 7.4% |
| Solar | 741 | 2.2% |
| Hydro | 781 | 2.3% |

#### **📊 Renewable Variability**
- **Range**: 18.4% to 77.1% (58.7 percentage point swing)
- **Volatility**: 14.2% standard deviation
- **UK exceeded 50% renewable for 49.2% of measured periods**
- Wind dominates renewable portfolio at **75.6%** of all renewable generation

#### **🕐 Temporal Patterns**
- **Peak renewable hour**: 2:00 AM (57.3%) - high wind, low demand
- **Lowest renewable hour**: 4:00 PM (45.0%) - evening demand surge
- **Most renewable day**: Saturday (55.7%) - lower industrial demand
- **Highest renewable single day**: January 1, 2026 (69.1%)

#### **💡 Key Insights**
1. **Wind has become the backbone of UK renewables** - offshore + onshore wind provides more generation than any other single source except gas
2. **Early morning hours are greenest** - combination of strong winds and low demand
3. **Grid is transitioning rapidly** - coal elimination complete, gas dependency declining
4. **Intermittency challenge remains** - 58.7 percentage point variation demonstrates need for storage/flexibility